# Preprocessing B984

This notebook visualizes and compiles the input data for the combined retrieval from flight B984.

In [ ]:
%load_ext autoreload
%autoreload 2
from joint_flight.notebook import *
from joint_flight import plot_observations
from joint_flight.data.b984 import *

## Overview

In [ ]:
from matplotlib.gridspec import GridSpec
plt.style.use("../misc/matplotlib_style.rc")
f = plt.figure(figsize=(15, 20))
gs = GridSpec(10, 3, width_ratios=[0.1, 1.0, 0.1])
axs = [f.add_subplot(gs[i, 1]) for i in range(10)]
legends = [f.add_subplot(gs[i, 2]) for i in range(10)]
names = [f.add_subplot(gs[i, 0]) for i in range(10)]


plot_observations(
    ISMAR, MARSS, RADAR, axs=axs,
    legends=legends,
    missing_channels=[14, 15, 16, 20],
    names=names
)
plt.show()

## Atmospheric background

In [ ]:
ATMOSPHERE

In [ ]:
from joint_flight import plot_atmosphere
from matplotlib.colors import Normalize, LogNorm

plot_atmosphere(RADAR, ATMOSPHERE)

# Obsrvations
## MARSS

In [ ]:
from matplotlib.colors import Normalize

def plot_channels(data, names):
    f, axs = plt.subplots(2, 1)
    
    x = RADAR.d / 1e3
    x_min = x.min()
    x_max = x.max()
    
    ax = axs[0]
    for d, n in zip(data, names):
        ax.plot(x, d, label=n)
    ax.legend(bbox_to_anchor=[1.0, 0.5])
    ax.set_xlim([x_min, x_max])
        
    ax = axs[1]
    y = RADAR.y
    x = RADAR.x / 1e3
    dbz = RADAR.dbz
    
    ax.pcolormesh(x, y / 1e3, dbz, norm=Normalize(-20, 20))
    ax.set_ylim(0, 11)
    ax.set_xlim([x_min, x_max])
    plt.show()

In [ ]:
MARSS

In [ ]:
n_rays = RADAR.d.size
y_marss = np.zeros((n_rays, 5))
y_marss_nedt = np.zeros((n_rays, 5))
tbs_marss = MARSS["brightness_temperatures"]
errors_marss = MARSS["errors"]

### 89 GHz

In [ ]:
data = [
    tbs_marss[:, 0],
]
names = map(lambda x: x.data, MARSS.channel[:1])
plot_channels(data, names)

In [ ]:
y_marss[:, 0] = tbs_marss[:, 0]
y_marss_nedt[:, 0] = errors_marss[:, 0] + 1e6 * SURFACE_MASK.data

### 157 GHz

In [ ]:
data = [
    tbs_marss[:, 1],
]
names = map(lambda x: x.data, MARSS.channel[1:2])
plot_channels(data, names)

In [ ]:
y_marss[:, 1] = tbs_marss[:, 1]
y_marss_nedt[:, 1] = errors_marss[:, 1] + 1e6 * SURFACE_MASK.data

### 183 GHz

In [ ]:
data = [
    tbs_marss[:, 2:],
]
names = map(lambda x: x.data, MARSS.channel[2:])
plot_channels(data, names)

In [ ]:
y_marss[:, 2:] = tbs_marss[:, 2:]
y_marss_nedt[:, 2:] = errors_marss[:, 2:]

## ISMAR

In [ ]:
y_ismar = np.zeros((n_rays, 10))
y_ismar_nedt = np.zeros((n_rays, 10))
tbs_ismar = ISMAR.brightness_temperatures.data
errors_ismar = ISMAR.errors.data

### 118 GHz

In [ ]:
data = [
    tbs_ismar[:, 0],
    tbs_ismar[:, 1],
    tbs_ismar[:, 2],
    tbs_ismar[:, 3],
    tbs_ismar[:, 4],
]
names = map(lambda x: x.data, ISMAR.channel[:5])
plot_channels(data, names)

In [ ]:
y_ismar[:, :5] = tbs_ismar[:, :5]
y_ismar_nedt[:, :5] = errors_ismar[:, :5]
y_ismar_nedt[:, 1:5] += SURFACE_MASK.data.reshape(-1, 1) * 1e6

### 243 GHz

In [ ]:
data = [
    tbs_ismar[:, 5],
    tbs_ismar[:, 6]
]
names = map(lambda x: x.data, ISMAR.channel[5:7])
plot_channels(data, names)

### 243 GHz - 30$^\circ$

In [ ]:
from joint_flight.data.b984 import ISMAR_30

tbs_ismar_30 = ISMAR_30["brightness_temperatures"].data
data = [
    tbs_ismar_30[:, 5],
    tbs_ismar_30[:, 6]
]
names = map(lambda x: x.data, ISMAR.channel[5:7])
plot_channels(data, names)

In [ ]:
y_ismar[:, 5] = 0.5 * (tbs_ismar[:, 5] + tbs_ismar[:, 6])
y_ismar_nedt[:, 5] = np.sqrt(0.5 * errors_ismar[:, 5] ** 2 + 0.5 * errors_ismar[:, 6] ** 2)

### 325 GHz

In [ ]:
data = [
    tbs_ismar[:, 7],
    tbs_ismar[:, 8],
    tbs_ismar[:, 9],
]
names = map(lambda x: x.data, ISMAR.channel[7:10])
plot_channels(data, names)

In [ ]:
y_ismar[:, 6:9] = tbs_ismar[:, 7:10]
y_ismar_nedt[:, 6:9] = errors_ismar[:, 7:10]

### 664 GHz

In [ ]:
data = [
    tbs_ismar[:, 17],
    tbs_ismar[:, 18],
]
names = map(lambda x: x.data, ISMAR.channel[17:19])
plot_channels(data, names)

In [ ]:
plt.plot(errors_ismar[:, 17])
plt.plot(errors_ismar[:, 18])
plt.show()

In [ ]:
y_ismar[:, 9] = 0.5 * (tbs_ismar[:, 17] + tbs_ismar[:, 18])
y_ismar_nedt[:, 9] = np.sqrt(0.5 * (errors_ismar[:, 17] ** 2 + errors_ismar[:, 18] ** 2))

In [ ]:
import xarray as xr
input_data = {
    # Atmosphere
    "latitude": (("rays",), RADAR.latitude),
    "longitude": (("rays",), RADAR.longitude),
    "altitude": (("rays", "levels",), np.broadcast_to(RADAR.height.data.reshape(1, -1),
                                                      RADAR.dbz.shape)),
    "pressure": (("rays", "levels"), ATMOSPHERE.pressure),
    "temperature": (("rays", "levels"), ATMOSPHERE.temperature),
    "cloud_water": (("rays", "levels"), ATMOSPHERE.cloud_water),
    #"H2O": (("rays", "levels"), ATMOSPHERE.relative_humidity),
    "H2O": (("rays", "levels"), ATMOSPHERE.water_vapor),
    "H2O_xa": (("rays", "levels"), ATMOSPHERE.relative_humidity),
    "O2": (("rays", "levels"), ATMOSPHERE.oxygen),
    "O3": (("rays", "levels"), ATMOSPHERE.ozone),
    "N2": (("rays", "levels"), ATMOSPHERE.nitrogen),
    # Surface variables
    "sea_surface_temperature": (("rays",), ATMOSPHERE.sea_surface_temperature),
    "wind_u": (("rays",), ATMOSPHERE.u10),
    "wind_v": (("rays",), ATMOSPHERE.v10),
    "surface_wind_speed": (("rays",
                            np.sqrt(ATMOSPHERE.u10 ** 2 +
                                    ATMOSPHERE.v10 ** 2))),
    # Observations
    "y_hamp_radar": (("rays", "range_bins"), RADAR.dbz),
    "y_hamp_radar_nedt": (("rays", "range_bins"), 0.5 * np.ones_like(RADAR.dbz)),
    "hamp_radar_range_bins": (("rays", "range_bin_boundaries"), 
                              np.broadcast_to(RADAR.range_bins.data.reshape(1, -1),
                                              (RADAR.dbz.shape[0], RADAR.dbz.shape[1] + 1))),
    "hamp_radar_sensor_pos": (("rays",), RADAR.range_bins.data[-1] * np.ones(RADAR.dbz.data.shape[0])),
    "y_marss": (("rays", "marss_channels"), y_marss),
    "y_marss_nedt": (("rays", "marss_channels"), y_marss_nedt),
    "marss_sensor_position": (("rays",), MARSS.altitude),
    "marss_sensor_line_of_sight": (("rays",), 180 * np.ones(MARSS.altitude.size)),
    "y_ismar": (("rays", "ismar_channels"), y_ismar),
    "y_ismar_nedt": (("rays", "ismar_channels"), y_ismar_nedt),
    "ismar_sensor_position": (("rays",), ISMAR.altitude),
    "ismar_sensor_line_of_sight": (("rays",), 180 * np.ones(ISMAR.altitude.size))
}
dataset = xr.Dataset(input_data)
dataset.to_netcdf("../data/input_b984.nc")
dataset.close()

In [ ]:
np.mean(np.diff(RADAR.x[:, 0]))